In [ ]:
!pip install tensorflow tensorflow-gpu pandas matplotlib scikit-learn

In [ ]:
#Import required Libraries
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
#Read data using Pandas
data = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [ ]:
#Preprocess the data
from tensorflow.keras.layers import TextVectorization

In [ ]:
X = data['comment_text']
y = data[data.columns[2:]].values

In [ ]:
#Number of word in the Vocabulary
MAX_FEATURES = 200000 

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [ ]:
#Teach the vectorizer the Vocabulary from our data
vectorizer.adapt(X.values)

In [ ]:
#Pass all our comment text to the vectorizer
vectorized_text = vectorizer(X.values)

In [ ]:
#Creating a TensorFlow data pipeline
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
#Partition the data
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
#Imports for Sequential Neural Networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer- GPU acceleration required for LSTM needs to have tanh activation function
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
#Train the model
history = model.fit(train, epochs=10, validation_data=val)

In [ ]:
#Plot the graph
from matplotlib import pyplot as plt
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
#Predict
input_text = vectorizer('Thank you for your help!')


In [ ]:
res=model.predict(np.expand_dims(input_text,axis=0))

In [ ]:
(res > 0.5).astype(int)

In [ ]:
#Evaluate the Model
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()


In [ ]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
#Save the model
model.save('toxicity.h5')

In [ ]:
#Integrate the model into Gradio App
!pip install gradio jinja2

In [ ]:
import gradio as gr

In [ ]:
#Load the model If used at a different environment
#model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(data.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=["text"],
                        outputs=["text"])

In [ ]:
interface.launch(share=True,show_error=True)